In [1]:
import numpy as np
import polars as pl
from tqdm import tqdm

from typing import List, Any

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from collections import Counter

In [2]:
from google.colab import drive
drive.mount('/content/drive')

PATH = 'drive/MyDrive/Colab Notebooks/ds-learning/lesson7'

Mounted at /content/drive


In [3]:
data = pl.read_parquet(f'{PATH}/train.parquet')
# датафрейм с обратными ребрами
data_rev = (
    data
    .rename({'uid': 'friend_uid', 'friend_uid': 'uid'})
    .select('uid', 'friend_uid')
)

# соединим все в один граф
data = pl.concat([data, data_rev])
data

uid,friend_uid
i64,i64
93464,114312
93464,103690
93464,108045
93464,116128
93464,94113
93464,101668
93464,118820
93464,93617
93464,97587


Данные состоят из двух колонок:

- `uid` – идентификатор пользователя
- `friend_uid` – идентификатор друга этого пользователя

Нашей задачей будет порекомендовать возможных друзей, для оценки вашего решения будет использоваться метрика Recall@10, равная проценту верно угаданных друзей

In [4]:
TOP_K = 20
RANDOM_STATE = 42

SUBMISSION_PATH = 'submission.parquet'


def user_intersection(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> int:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: number of items in intersection of y_rel and y_rec (truncated to top-K)
    """
    return len(set(y_rec[:k]).intersection(set(y_rel)))


def user_recall(y_rel: List[Any], y_rec: List[Any], k: int = 10) -> float:
    """
    :param y_rel: relevant items
    :param y_rec: recommended items
    :param k: number of top recommended items
    :return: percentage of found relevant items through recommendations
    """
    return user_intersection(y_rel, y_rec, k) / min(k, len(set(y_rel)))

## Валидация

Так как у нас нет временной последовательности и рекомендации друзей не так сильно зависят от временной составляющей, в качестве можно использовать случайно выбранные ребра в графе (при этом для каждого пользователя будет равная пропорция друзей в валидации, которую можно достичь с помощью stratify параметра)

In [5]:
# зафиксируем генератор случайных чисел
random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

In [6]:
# отфильтруем тех пользователей, у которых только один друг :(
# для того, чтобы в тренировочной выборке и валидации было хотя бы по одному другу
friends_count = data.group_by('uid').count()
filtered_uid = set(friends_count.filter(pl.col('count') > 1)['uid'].to_list())

data_filtered = data.filter(pl.col('uid').is_in(filtered_uid))

# случайно выбираем ребра для валидационной выборки
train_df, test_df = train_test_split(
    data_filtered.filter(pl.col('uid').is_in(filtered_uid)),
    stratify=data_filtered['uid'],
    test_size=0.1,
    random_state=RANDOM_STATE
)

train_df

uid,friend_uid
i64,i64
62053,63575
31895,59356
97127,32271
89,11703
105178,47188
116127,52662
33824,15235
23690,103992
94660,45709


## Бейзлайн (Random)

In [7]:
grouped_df = (
    test_df
    .group_by('uid')
    .agg(pl.col('friend_uid').alias('y_rel'))
    .join(
        train_df
        .group_by('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

median_seq_len = int(grouped_df['user_history'].map_elements(len).median())
print(f"среднее число uid в user_history: {median_seq_len}")

среднее число uid в user_history: 36


In [ ]:
n_users = train_df['uid'].max() + 1

# количество друзей у каждого пользователя
friends_count = np.zeros(n_users)
for uid, count in Counter(train_df['uid']).items():
    friends_count[uid] = count

friends_count /= sum(friends_count)

In [ ]:
recall_list = []
recs = np.random.choice(n_users, size=(n_users, TOP_K + median_seq_len), p=friends_count)

for user_id, y_rel, user_history in tqdm(grouped_df.rows()):
    y_rec = [uid for uid in recs[user_id] if uid not in user_history]
    recall_list.append(user_recall(y_rel, y_rec))

print(f'Recall@{TOP_K} = {np.mean(recall_list)}')

100%|██████████| 92562/92562 [00:17<00:00, 5346.50it/s]


Recall@20 = 0.00030239276771330966


# Второй бейзлайн (Adamic-Adar)

In [7]:
adjacency_matrix = sp.coo_array(
    (np.ones(train_df.shape[0], dtype=int),
    (train_df['uid'], train_df['friend_uid'])),
    shape=(data_filtered.shape[0], data_filtered.shape[0])
).tocsr()

In [8]:
def get_adamic_adar_matrix(adjacency_matrix):
    degree_vector = np.sum(adjacency_matrix, axis=1).reshape(1, -1)
    log_degree_vector = np.log(degree_vector)
    log_degree_vector[log_degree_vector == 0.0] = np.inf
    resource_matrix = adjacency_matrix / log_degree_vector
    return resource_matrix @ adjacency_matrix

In [9]:
aa_matrix = get_adamic_adar_matrix(adjacency_matrix)

<ipython-input-8-e5d6d03f8e8b>:3: RuntimeWarning: divide by zero encountered in log
  log_degree_vector = np.log(degree_vector)


In [ ]:
# Проверка метрики, если сохраняли историю

recall_list = []

for user_id, y_rel, user_history in tqdm(grouped_df.rows()):
    ind = np.argpartition(aa_matrix[[user_id]].data, -TOP_K)[-TOP_K:]
    recs = aa_matrix[[user_id]].indices[ind]
    y_rec = [uid for uid in recs if uid not in user_history]
    recall_list.append(user_recall(y_rel, y_rec, TOP_K))

print(f'Recall@{TOP_K} = {np.mean(recall_list)}')

100%|██████████| 92562/92562 [00:55<00:00, 1672.46it/s]

Recall@20 = 0.17930244849953691


## Построим рекомендации

In [10]:
sample_submission = pl.read_parquet(f'{PATH}/sample_submission.parquet')

grouped_df = (
    sample_submission.select('uid')
    .join(
        train_df
        .group_by('uid')
        .agg(pl.col('friend_uid').alias('user_history')),
        'uid',
        how='left'
    )
)

submission = []

K = 100
for user_id, user_history in tqdm(grouped_df.rows()):
    user_history = [] if user_history is None else user_history
    user_history.append(user_id)
    data_len = aa_matrix[[user_id]].data.shape[0]
    ind = np.argpartition(aa_matrix[[user_id]].data, -min(K, data_len))[-K:]
    recs = aa_matrix[[user_id]].indices[ind]
    rec_pairs = [(aa_ind, uid) for aa_ind, uid in zip(aa_matrix[[user_id]].data[ind],
                                                      aa_matrix[[user_id]].indices[ind])]
    rec_pairs.sort(reverse=True)
    y_rec = [uid for _, uid in rec_pairs if uid not in user_history]
    submission.append((user_id, y_rec[:20]))


submission = pl.DataFrame(submission, schema=['user_id', 'y_recs'])

submission.write_parquet('submission_aa_v3.parquet')
submission

100%|██████████| 85483/85483 [02:17<00:00, 621.82it/s]


user_id,y_recs
i64,list[i64]
0,"[68034, 43989, … 64886]"
1,"[90756, 92413, … 116400]"
3,"[70188, 94808, … 118664]"
4,"[38464, 36234, … 95849]"
5,"[18562, 48984, … 80655]"
6,"[32090, 103202, … 104824]"
7,"[102537, 76321, … 10495]"
8,"[47998, 98545, … 52138]"
9,"[10855, 29133, … 112158]"


In [11]:
# проверка "заполненности" рекомендаций

submission['y_recs'].map_elements(lambda x : len(x) >= 20).value_counts()

y_recs,count
bool,u32
false,475
true,85008
